## Check MPI & IPyParallel setup

First we import parallel python and ipcmagic which allows us to spinup a cluster of Python engines within an MPI context.

In [1]:
import numpy as np
import ipcmagic
import ipyparallel as ipp
import matplotlib
import matplotlib.pyplot as plt

We now start a cluster of 4 workers (MPI ranks). This might take a while to finish.

**Important: This cell should only be executed once at kernel startup. If you re-execute everything in the notebook, this cell should be skipped. The only time this cell should be executed is immediately after re-starting the kernel or after having stopped the cluster with `ipcluster stop`.**

In [2]:
%ipcluster start -n 4 --mpi

IPCluster is ready! (7 seconds)


The desired output here should be:
```
IPCluster is ready! (XX seconds)
```
If you get a different message, something is wrong in your setup. Try again. If the problem persists, reach out to us and we will try and help get things fixed!

In [3]:
rc = ipp.Client()
rc.ids
dv = rc[:]
dv.activate()
dv.block = True
print("Running IPython Parallel on {0} MPI engines".format(len(rc.ids)))
%px from mpi4py import MPI
%px mpi_rank = MPI.COMM_WORLD.Get_rank()
dv_master = dv.pull('mpi_rank').index(0)

Running IPython Parallel on 4 MPI engines


The desired output here should be:
```
Running IPython Parallel on 4 MPI engines
```

In [4]:
%%px
import time
import timeit
import numpy as np
from mpi4py import MPI

Since we enabled `%%px` for this cell, the commands will be executed on *all* of the workers. Let's import the mpi4py package as MPI and then print the number of ranks and the rank ID on every worker.

In [5]:
%%px
from mpi4py import MPI

comm = MPI.COMM_WORLD
mpi_size = comm.Get_size()
mpi_rank = comm.Get_rank()

print(mpi_size)
print(mpi_rank)

[stdout:0] 
4
3
[stdout:1] 
4
1
[stdout:2] 
4
2
[stdout:3] 
4
0


Since we are receiving standard output from each of the ranks, it is marked using <tt>[stdout:NN]</tt> where <tt>NN</tt> is the number of the rank.

If we do not enable `%%px` we are no longer sending commands to the workers, but we will be executing commands on frontend again.

In [6]:
print(10)

10


Re-enabling `%%px` again, we are executing commands to the workers again. 

In [7]:
%%px
print(10)

[stdout:0] 10
[stdout:1] 10
[stdout:2] 10
[stdout:3] 10


In order to stop the cluster, we need to make sure that we are executing commands on the frontend and then issueing the `%ipcluster stop` command.

In [8]:
%ipcluster stop